In [1]:
%pip install Ollama -q
%pip install llama_index.llms.ollama -q
%pip install llama_index.core -q
%pip install llama_index.embeddings.ollama -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.ollama import OllamaEmbedding

In [3]:
import ollama
import requests
from pathlib import Path
import re
import nltk
import json
import numpy as np

In [4]:
#TO-DO: Get a list of the urls corresponding to each link text file

In [5]:
# Iterate over all items in the directory and create a list of pathlib.PosixPath entries each detailing a path to a .txt training file
#.txt files are expected to be numered

def get_training_files(path):

   directory = Path(path)
   training_files = []

   for file in directory.iterdir():
      training_files.append(file)

   # Sort the list of pathlib.PosixPath entries based on numeric part of the .txt file
   """""
   1) files.sort(: This starts the sorting operation on the files list. The sort() method sorts the elements of the list in place.
   
   2) key=lambda x:: This is a key function used to customize the sorting order. Here, lambda x: defines an anonymous function that takes one argument x.
   
   3) re.search(r'\d+', x.stem): This part uses the re.search() function from the re module to search for a pattern in the file name.
      \d+ is a regular expression pattern that matches one or more digits. x.stem gives the base name of the file without the extension.

   4) .group(): This method returns the matched part of the string. In this case, it returns the matched digits as a string.
   
   5) int(...): This converts the matched string of digits to an integer. This is necessary because we want to sort the files numerically,
      not lexicographically.

   Putting it all together, this line sorts the list of files (files) based on the numeric part extracted from each file's name.
   The lambda function extracts this numeric part using a regular expression and converts it to an integer, which is then used as the sorting key.
"""""
   training_files.sort(key=lambda x: int(re.search(r'\d+', x.stem).group()))

   return training_files

training_set = get_training_files("/Users/raulriveromartinez/Desktop/Skyy AI/training_data")


<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
/var/folders/wd/84g4f7_55r39pnh89vfg3s480000gn/T/ipykernel_11323/4130321290.py:13: SyntaxWarning: invalid escape sequence '\d'
  """""


In [6]:
#Generate a list of str that contains only the names of the .txt files without their full directory

def get_filenames(training_files):   

   filenames = []
   
   for file in training_files:
      filenames.append(file.name)

   return filenames

filenames_set = get_filenames(training_set)

In [7]:
def get_files_contents(training_files):
     
     # Create an empty list to store the contents of each text file in the training dataset
     files_contents = []

     # Iterate through each text file in the training dataset and add its contents to the files_contents list
     for file in training_files:
          with open(file, "r") as contents:
               txt_contents = contents.read()
               files_contents.append(txt_contents)

     return files_contents

contents_set = get_files_contents(training_set)

In [8]:
#put the filename list and the file_content lists on a dictionary with two key-value pairs: filename and contents

def get_contents_dict(filenames, files_contents):

    AllData = []
    x = 0
    for i in range(0,len(filenames)):
        data = {'filename':filenames[i],'content':files_contents[i]}
        AllData.append(data)

    return AllData

files_content_dicts = get_contents_dict(filenames_set, contents_set)

In [9]:
# allData is the name of the dictionary storing the filename and contents of each file as key value pairs

def chunk_data_as_sentences(allData):
	# list to store all data as individual sentences with their corresponding filename
	system_data = list()
	# process each file's content in the dictionary
	for i in allData:
		# segment (chunk) each file's contents into individual sentences
		sentences = nltk.sent_tokenize(i['content'])
		# store each sentence with its filename
		for s in sentences:
			system_data.append({'filename': i['filename'], 'content': s})
			
	return system_data

chunked_files_content_dicts = chunk_data_as_sentences(files_content_dicts)


In [10]:
# embedding function to convert sentence into vector

def set_local_model(model_name = "llama3"):
    #TO-DO:add script to check if model is installed
    local_model = OllamaEmbedding(model_name = model_name, base_url = 'http://localhost:11434')
    return local_model

local_model = set_local_model("llama3")

In [ ]:
system_record = list()

for i in chunked_files_content_dicts[:5]:
		# function call need to match completed implementation
		embedding = local_model.get_query_embedding(i['content'])
		record = {'filename': i['filename'], 'content': i['content'], 'vector': embedding}
		system_record.append(record)
		
for i in system_record: print(i) 

In [ ]:
# create embeddings for the "chunked" data
# pass allData as dictionary

def embed_sentences(allData):
	# create a list to store each sentence in each file with its
	# filename, the sentence text, and its embedding
	system_record = list()
	for i in allData:
		# function call need to match completed implementation
		embedding = local_model.get_query_embedding(i['content'])
		record = {'filename': i['filename'], 'content': i['content'], 'vector': embedding}
		system_record.append(record)

	with open('mdc_index.json', 'w') as outfile:
        	json.dump(record, outfile, indent=2)

embed_sentences(chunked_files_content_dicts)